In [1]:
from astropy.table import Table

import glob

import numpy as np

In [4]:
#master_table = Table.read('../spirals/Pipe3D-master_file_vflag_10_smooth2p27.txt', format='ascii.ecsv')
master_table = Table.read('../spirals/Pipe3D-master_file_vflag_10_smooth2p27_N2O2_noWords.txt', 
                          format='ascii.commented_header')

## Galaxies with rotation curves

In [5]:
#boolean = np.logical_or(master_table['curve_used'] == 'non', master_table['curve_used'] == 'none')
boolean = master_table['curve_used'] == -99

rotCurve_galaxies = master_table[np.logical_not(boolean)]

print('Galaxies in MaNGA DR15:', len(master_table))
print('Galaxies with rotation curves:', len(rotCurve_galaxies))

Galaxies in MaNGA DR15: 4815
Galaxies with rotation curves: 2426


## Galaxies in each environment

In [6]:
vboolean = rotCurve_galaxies['vflag'] == 1
wboolean = rotCurve_galaxies['vflag'] == 0

void_table = rotCurve_galaxies[vboolean]
wall_table = rotCurve_galaxies[wboolean]
edge_table = rotCurve_galaxies[np.logical_not(np.logical_or(vboolean, wboolean))]

print('Void galaxies with rotation curves:', len(void_table))
print('Wall galaxies with rotation curves:', len(wall_table))
print('Edge galaxies with rotation curves:', len(edge_table))

Void galaxies with rotation curves: 642
Wall galaxies with rotation curves: 938
Edge galaxies with rotation curves: 846


## Galaxies by magnitude

In [7]:
print('Dwarf void galaxies:', sum(void_table['rabsmag'] >= -17))
print('Dwarf wall galaxies:', sum(wall_table['rabsmag'] >= -17))

print('-17:-18 void galaxies:', sum( np.logical_and( void_table['rabsmag'] < -17, void_table['rabsmag'] >= -18)))
print('-17:-18 wall galaxies:', sum( np.logical_and( wall_table['rabsmag'] < -17, wall_table['rabsmag'] >= -18)))

print('Faint void galaxies (fainter than -18):', sum(void_table['rabsmag'] >= -18))
print('Faint wall galaxies (fainter than -18):', sum(wall_table['rabsmag'] >= -18))

print('-18:-19 void galaxies:', sum( np.logical_and( void_table['rabsmag'] < -18, void_table['rabsmag'] >= -19)))
print('-18:-19 wall galaxies:', sum( np.logical_and( wall_table['rabsmag'] < -18, wall_table['rabsmag'] >= -19)))

print('-19:-20 void galaxies:', sum( np.logical_and( void_table['rabsmag'] < -19, void_table['rabsmag'] >= -20)))
print('-19:-20 wall galaxies:', sum( np.logical_and( wall_table['rabsmag'] < -19, wall_table['rabsmag'] >= -20)))

print('Void galaxies brighter than -20:', sum( void_table['rabsmag'] < -20))
print('Wall galaxies brighter than -20:', sum( wall_table['rabsmag'] < -20))

print('-20:-21 void galaxies:', sum( np.logical_and( void_table['rabsmag'] < -20, void_table['rabsmag'] >= -21)))
print('-20:-21 wall galaxies:', sum( np.logical_and( wall_table['rabsmag'] < -20, wall_table['rabsmag'] >= -21)))

print('Void galaxies brighter than -21:', sum( void_table['rabsmag'] < -21))
print('Wall galaxies brighter than -21:', sum( wall_table['rabsmag'] < -21))

Dwarf void galaxies: 4
Dwarf wall galaxies: 1
-17:-18 void galaxies: 77
-17:-18 wall galaxies: 78
Faint void galaxies (fainter than -18): 81
Faint wall galaxies (fainter than -18): 79
-18:-19 void galaxies: 167
-18:-19 wall galaxies: 215
-19:-20 void galaxies: 166
-19:-20 wall galaxies: 243
Void galaxies brighter than -20: 228
Wall galaxies brighter than -20: 401
-20:-21 void galaxies: 140
-20:-21 wall galaxies: 217
Void galaxies brighter than -21: 88
Wall galaxies brighter than -21: 184


## Fraction of masked spaxels

In [13]:
sum_frac_masked_spaxels = 0
N_gal = 0

for file in glob.iglob('../spirals/Pipe3D-rot_curve_data_files/*_gal_stat_data.txt'):
    
    # Extract plate, fiberID
    file_name = file.split('/')
    components = file_name[-1].split('_')
    plate, fiberID = components[0].split('-')
    
    # Find galaxy in master_table
    plate_indices = np.argwhere(master_table['MaNGA_plate'] == int(plate))
    fiber_idx = np.where(master_table['MaNGA_IFU'][plate_indices] == int(fiberID))
    gal_idx = plate_indices[fiber_idx]
    
    if master_table['curve_used'][gal_idx] != 'non' and master_table['curve_used'][gal_idx] != 'none':
        
        # Read in data
        gal_stats = Table.read(file, format='ascii.ecsv')

        # Update number of galaxies
        N_gal += 1

        # Update sum of the fraction of the number of masked spaxels
        sum_frac_masked_spaxels += gal_stats['frac_masked_spaxels'][0]
        
        
print('Average number of masked spaxels:', sum_frac_masked_spaxels/N_gal)

/Users/kellydouglass/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1020: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)


Average number of masked spaxels: 0.2509300974382464


# $\chi^2$ 2-sample test

A $\chi^2$ two sample test on each subset of the galaxies to determine the likelihood that they come from the same distribution.